In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGANGP/200/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGANGP/200/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000938', 'ENSG00000005844', 'ENSG00000009790',
       'ENSG00000013441', 'ENSG00000019582', 'ENSG00000028137',
       'ENSG00000038427', 'ENSG00000044574', 'ENSG00000049860',
       'ENSG00000051523',
       ...
       'ENSG00000204287', 'ENSG00000211772', 'ENSG00000211899',
       'ENSG00000223553', 'ENSG00000231500', 'ENSG00000240356',
       'ENSG00000244734', 'ENSG00000257207', 'ENSG00000269028', 'label'],
      dtype='object', length=115)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000938,ENSG00000005844,ENSG00000009790,ENSG00000013441,ENSG00000019582,ENSG00000028137,ENSG00000038427,ENSG00000044574,ENSG00000049860,ENSG00000051523,...,ENSG00000204287,ENSG00000211772,ENSG00000211899,ENSG00000223553,ENSG00000231500,ENSG00000240356,ENSG00000244734,ENSG00000257207,ENSG00000269028,label
VUMC-HC-0033-TR2591,4.432779,4.211676,4.252125,4.328301,7.136091,3.920710,3.720018,4.252125,4.432779,4.643048,...,5.318681,4.843824,4.911210,5.112603,6.828857,3.535280,6.995484,9.847484,7.757440,HC
Vumc-HD-70-TR1062,5.786592,5.247890,5.983595,5.114879,9.178061,4.546459,3.951486,5.653279,5.505492,6.338547,...,7.435968,6.744823,6.633504,3.522177,9.405474,3.701701,10.021953,8.262227,6.645036,HC
VUMC-HC0053-DOT-HD-48h-TR3087,5.526849,4.934939,5.442502,4.467036,7.631874,4.292393,3.535847,5.114817,5.038021,5.593316,...,5.631686,5.894780,4.559495,4.030369,7.399434,7.783065,7.050183,8.831693,4.253528,HC
Vumc-HD-149-TR932,4.923042,3.937986,4.556107,4.275620,6.897237,3.808559,3.647372,4.492627,3.083092,4.853377,...,5.606988,5.466326,4.389800,5.839804,8.609324,9.503594,8.661739,3.647372,8.517492,HC
Vumc-HD-36-TR1165,6.484772,5.938859,7.417415,6.564767,8.796382,5.339481,5.020424,3.862107,4.172174,7.306126,...,6.780833,7.808699,6.525333,3.862107,10.296126,5.671874,9.877834,3.862107,7.025844,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,3.191670,3.109249,3.453991,3.122966,5.473135,3.101408,3.083092,4.557811,3.084079,3.161223,...,3.406650,3.123706,3.150261,3.083876,6.750561,11.260020,8.951130,7.743332,4.050507,OC
77,4.644648,4.909522,5.300314,5.949164,8.282188,4.654642,3.223228,5.035921,4.874362,5.935729,...,5.493013,4.501245,4.583561,5.434829,8.040750,6.102728,9.202395,8.059603,6.978441,OC
78,4.765028,5.201696,4.836886,6.338811,8.240970,5.413673,3.258814,5.280987,4.610653,6.282280,...,6.939704,4.001895,5.075729,5.402220,7.164021,6.535313,9.381407,7.811015,7.032177,OC
79,5.026990,4.840318,4.974492,5.400462,8.879166,4.921508,3.279961,4.974441,4.753414,6.272022,...,6.504180,3.980940,5.192759,4.685068,7.922683,6.275969,9.461527,7.186731,6.546957,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

          HC       0.91      0.91      0.91       131
          OC       0.78      0.78      0.78        54

    accuracy                           0.87       185
   macro avg       0.84      0.84      0.84       185
weighted avg       0.87      0.87      0.87       185

[[119  12]
 [ 12  42]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

          HC       0.89      0.91      0.90       131
          OC       0.77      0.74      0.75        54

    accuracy                           0.86       185
   macro avg       0.83      0.82      0.83       185
weighted avg       0.86      0.86      0.86       185

[[119  12]
 [ 14  40]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7891891891891892
              precision    recall  f1-score   support

          HC       0.82      0.89      0.86       131
          OC       0.67      0.54      0.60        54

    accuracy                           0.79       185
   macro avg       0.75      0.72      0.73       185
weighted avg       0.78      0.79      0.78       185

[[117  14]
 [ 25  29]]
Dokładność: 0.7837837837837838
              precision    recall  f1-score   support

          HC       0.83      0.87      0.85       131
          OC       0.65      0.57      0.61        54

    accuracy                           0.78       185
   macro avg       0.74      0.72      0.73       185
weighted avg       0.78      0.78      0.78       185

[[114  17]
 [ 23  31]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8540540540540541
              precision    recall  f1-score   support

          HC       0.84      0.98      0.91       131
          OC       0.94      0.54      0.68        54

    accuracy                           0.85       185
   macro avg       0.89      0.76      0.79       185
weighted avg       0.87      0.85      0.84       185

[[129   2]
 [ 25  29]]
Dokładność: 0.8486486486486486
              precision    recall  f1-score   support

          HC       0.83      0.98      0.90       131
          OC       0.93      0.52      0.67        54

    accuracy                           0.85       185
   macro avg       0.88      0.75      0.78       185
weighted avg       0.86      0.85      0.83       185

[[129   2]
 [ 26  28]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       131
           1       0.83      0.70      0.76        54

    accuracy                           0.87       185
   macro avg       0.86      0.82      0.84       185
weighted avg       0.87      0.87      0.87       185

[[123   8]
 [ 16  38]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       131
           1       0.83      0.65      0.73        54

    accuracy                           0.86       185
   macro avg       0.85      0.80      0.82       185
weighted avg       0.86      0.86      0.85       185

[[124   7]
 [ 19  35]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.8864864864864865
              precision    recall  f1-score   support

          HC       0.90      0.95      0.92       131
          OC       0.85      0.74      0.79        54

    accuracy                           0.89       185
   macro avg       0.87      0.84      0.86       185
weighted avg       0.88      0.89      0.88       185

[[124   7]
 [ 14  40]]
Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

          HC       0.90      0.92      0.91       131
          OC       0.80      0.74      0.77        54

    accuracy                           0.87       185
   macro avg       0.85      0.83      0.84       185
weighted avg       0.87      0.87      0.87       185

[[121  10]
 [ 14  40]]
